Lucas-Kanade Optical Flow Tracking with Shi-Tomasi Corner Detection

In [ ]:
import cv2
import numpy as np
import time

class LucasKanade_OpticalFlow:
    def __init__(self, video_path, window_width=1280, window_height=720, 
                 max_corners=1000, quality_level=0.01, min_distance=0.1):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(self.video_path)

        # Get the frame width and height from the video 
        self.frame_width = int(self.cap.get(3))
        self.frame_height = int(self.cap.get(4))

        # Parameters of Shi-Tomasi Corner Detection
        self.max_corners = max_corners
        self.quality_level = quality_level
        self.min_distance = min_distance
        self.feature_params = dict(maxCorners=max_corners, qualityLevel=quality_level, minDistance=min_distance)

        # Parameters of Lucas-Kanade Optical Flow
        self.lk_params = dict(winSize=(25, 25), maxLevel=3,
                              criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,
                                        30, 0.01))

        # Initialize Variables for LK
        self.prev_frame = None
        self.prev_gray = None
        self.prev_corners = None

        # Set window size and position (1280x720 center screen)
        self.window_width = window_width
        self.window_height = window_height
        self.position_window()

    def position_window(self):
        cv2.namedWindow('Feature Tracking', cv2.WINDOW_NORMAL)
        cv2.moveWindow('Feature Tracking', (1920 - self.window_width) // 2, (1080 - self.window_height) // 2)
        cv2.resizeWindow('Feature Tracking', self.window_width, self.window_height)

    def detect_corners(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners = cv2.goodFeaturesToTrack(gray, mask=None, **self.feature_params)
        return corners, gray

    def track_features(self):
        # Read the first frame
        ret, self.prev_frame = self.cap.read()
        self.prev_corners, self.prev_gray = self.detect_corners(self.prev_frame)
        # Create a mask for drawing purposes
        mask = np.zeros_like(self.prev_frame)
        start_time = time.time()
        frame_count = 0

        while True:
            ret, frame = self.cap.read()
            if not ret:
                break
            # Detect features in the current frame using Shi-Tomasi corner detection
            new_corners, gray = self.detect_corners(frame)
            # Calculate optical flow using Lucas-Kanade
            new_corners, status, err = cv2.calcOpticalFlowPyrLK(self.prev_gray, gray, self.prev_corners, None,
                                                               **self.lk_params)
            # Select good points
            good_new = new_corners[status == 1]
            good_old = self.prev_corners[status == 1]
            # Draw tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
                frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)

            img = cv2.add(frame, mask)

            cv2.imshow('Feature Tracking', img)
            k = cv2.waitKey(30) & 0xff
            if k == 27 or k == ord('q'):  # 'q' key to quit
                break

            # Update previous frame and points
            self.prev_gray = gray.copy()
            self.prev_corners = good_new.reshape(-1, 1, 2)

            # Display processing time and fps
            frame_count += 1
            if frame_count % 10 == 0:
                elapsed_time = time.time() - start_time
                fps = frame_count / elapsed_time
                print(f"Processing Time: {elapsed_time:.2f}s, FPS: {fps:.2f}")

        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "D:\MAPUA\Thesis\SampleVideos\SMTracking_LongSleeve.MOV"
    feature_tracker = LucasKanade_OpticalFlow(video_path, window_width=1280, window_height=720)
    feature_tracker.track_features()


Lucas-Kanade Optical Flow with Mediapipe Pose for Joint Detection